In [1]:
from utils import *

In [2]:
# load data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/")

Extracting tmp/train-images-idx3-ubyte.gz
Extracting tmp/train-labels-idx1-ubyte.gz
Extracting tmp/t10k-images-idx3-ubyte.gz
Extracting tmp/t10k-labels-idx1-ubyte.gz


With all these layers, we have to pass the same arguments over and over again. One way to approach this is to take advantage of the Python function partial. Partial basically allows you to specify a set of parameters to use for a function every time. You can leave other parameters unspecified and pass them on call. Here's an example that adds 5 numbers together. Suppose we want to call this function many times with 4 known values and 1 that changes at every call.

In [3]:
from functools import partial

def addbunch(a, b, c, d, e):
    return a+b+c+d+e

addbunch(1, 2, 3, 4, 5)

15

In [7]:
addbunchmod = partial(addbunch, b=2, c=3, d=4, e=5)
print(addbunchmod(1))
print(addbunchmod(2))
print(addbunchmod(3))

15
16
17


Each time we called the new function `addbunchmod` it had default values for b, c, d, and e. 

We can use this to streamline the previous notebook's Batch Normalization code.

In [8]:
from functools import partial

reset_graph()

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

learning_rate = 0.01
batch_norm_momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
training = tf.placeholder_with_default(False, shape=(), name='training')

with tf.name_scope("dnn"):
    he_init = tf.contrib.layers.variance_scaling_initializer()
    
    # using partial to make layer functions that have repetitive parameters set
    my_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=batch_norm_momentum)
    my_dense_layer = partial(tf.layers.dense, kernel_initializer=he_init)
    
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
    hidden2 = my_dense_layer(bn1, n_hidden2, name="hidden2")
    bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
    logits_before_bn = my_dense_layer(bn2, n_outputs, name="outputs")
    logits = my_batch_norm_layer(logits_before_bn)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y , 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()

In [9]:
# execution

n_epochs = 40
batch_size = 50

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op, extra_update_ops], feed_dict={training: True, X:X_batch, y:y_batch})
        if epoch % 5 == 0 or epoch == n_epochs - 1:
            acc_train = accuracy.eval(feed_dict={X:mnist.train.images, y:mnist.train.labels})
            acc_val = accuracy.eval(feed_dict={X:mnist.validation.images, y:mnist.validation.labels})
            print(epoch, "train acc:", acc_train, "val acc:", acc_val)
    acc_test = accuracy.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels})
    print("Test acc:", acc_test)

0 train acc: 0.914236 val acc: 0.9192
5 train acc: 0.972 val acc: 0.9666
10 train acc: 0.984836 val acc: 0.9744
15 train acc: 0.990818 val acc: 0.9788
20 train acc: 0.993927 val acc: 0.9802
25 train acc: 0.995945 val acc: 0.9806
30 train acc: 0.996909 val acc: 0.9806
35 train acc: 0.998236 val acc: 0.9792
39 train acc: 0.998655 val acc: 0.9834
Test acc: 0.9803
